In [7]:
import requests
from requests.auth import HTTPBasicAuth
import json
import os
import sys
sys.path.append(os.path.abspath(".."))
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
base = os.getenv("INTERVISTA_URL")
path = os.getenv("INTERVISTA_PATH")
user= os.getenv("INTERVISTA_USER")
pwd= os.getenv("INTERVISTA_PWD")

In [3]:
pv_ids = [
    "64",
    "3415",
    "3424",
    "3436",
    "3448",
    "3571",
    "3610",
]

# Get product configurations from intervista backend

In [181]:
data = []
for pv_id in pv_ids:
    url = f"{base}{pv_id}{path}"
    session = requests.Session()
    response = session.get(url, auth=HTTPBasicAuth(user, pwd))
    data.append(response.json()['result'][0])

print(json.dumps(data, indent=2, ensure_ascii=False))

[
  {
    "product": {
      "productId": 46,
      "productVariantId": 64,
      "name": "FirmenSchutzbrief (Standardtarif)",
      "description": "FirmenSchutzbrief by ELEMENT",
      "versionInfo": "FirmenSchutzbrief by ELEMENT",
      "displayName": "FirmenSchutzbrief by ELEMENT",
      "tooltip": null,
      "state": "Offen für den Bestand",
      "typeId": -101001,
      "availableFrom": 1614902400000,
      "availableTo": null,
      "personSpecific": false,
      "providerId": 10,
      "providerName": "ELEMENT",
      "providerShortName": "EL",
      "tag": [
        "Versicherung",
        "Geschäftskundentarif",
        "FirmenSchutz",
        "Einzelversicherung",
        "Multigerätetarif"
      ],
      "tagrenderdata": null,
      "rule": [
        "de.schutzgarant.core.plugins.FirmenSchutzPlausiPlugin",
        "de.schutzgarant.core.plugins.GesamtVersicherungssummenBerechner",
        "at.intervista.collactive.sellactive.plugin.plausi.RelatedDocumentCheckerPlugin",
    

In [182]:
from pydantic import BaseModel, Field, ValidationError, model_validator, field_validator
from typing import Optional, List

class PaymentProvider(BaseModel):
    providerType: int = Field(..., description="Type of payment service provider. Possible values: -160 = Credit card, -161 = Bank, -162 = Amazon Pay, -163 = PayPal, -165 = Google Pay")
    providerNumber: Optional[str] = Field(None, description="BIC of the bank, necessary for intern'ational accounts only")
    providerName: Optional[str] = Field(None, description="Possible values: Name of bank, 'VISA', 'AMEX', ...")
    accountNumber: Optional[str] = Field(None, description="IBAN, credit card number or payment id (associated email for online payment services)")
    accountOwner: Optional[str] = Field(None, description="Name of account owner")
    expirationDate: Optional[str] = Field(None, description="Needed when providerType=-160")

    @model_validator(mode="after")
    def check_fields(self):
        ptype = getattr(self, 'providerType')
        if ptype == -160: 
            required = ['providerName', 'accountNumber', 'accountOwner', 'expirationDate']
        elif ptype == -161:
            required = ['providerNumber', 'accountNumber', 'accountOwner']
        elif ptype < -161 and ptype > -166:
            required = ['accountNumber']
        else:
            raise ValueError(f"ProviderType {ptype} unknown. Possible values: -160 = Credit card, -161 = Bank, -162 = Amazon Pay, -163 = PayPal, -165 = Google Pay")

        missing = [f for f in required if not getattr(self, f)]
        if missing:
            raise ValueError(f"Missing required fields for providerType {ptype}: {', '.join(missing)}")
        return self

class Address(BaseModel):
    typeId: int = Field(..., description="Address type. -10 = Private, -11 = Business, -12 = Billing, -13 = Delivery")
    street: str = Field(..., description="Street (without house number)")
    streetNumber: str = Field(..., description="House number")
    addition: Optional[str] = Field(None, description="Additional info (e.g. '2nd floor')")
    zipcode: str = Field(..., description="Postal code")
    city: str = Field(..., description="City name")
    countryCode: str = Field(..., min_length=2, max_length=2, description="2-char ISO country code (e.g. DE)")

class ContactInfo(BaseModel):
    typeId: int = Field(..., description="Contact type. -21=Mobile, -20=Landline, -23=Email, -22=Fax")
    value: str = Field(..., description="Contact value: phone/email/etc.")

class Person(BaseModel):
    typeId: int = Field(..., description="Person type: -2101 main, -30 company")
    salutationId: Optional[int] = Field(None, description="Salutation ID, required for individuals")
    title: Optional[str] = Field(None, description="Title such as Dr., Prof., etc.")
    firstName: Optional[str] = Field(None, description="First name. Not for companies.")
    lastName: str = Field(..., description="Last name or company name")
    dateOfBirth: Optional[str] = Field(None, description="Date of birth (YYYY-MM-DD)", pattern="^\\d{4}-\\d{2}-\\d{2}$")
    addresses: List[Address] = Field(default_factory=list, description="Addresses of the person")
    contactInformations: List[ContactInfo] = Field(default_factory=list, description="Contact info (phone, email, etc.)")

    @model_validator(mode="before")
    @classmethod
    def check_lists(cls, data):
        errors = []
        if "addresses" in data and not isinstance(data["addresses"], list):
            errors.append(f"addresses must be a list, got {type(data['addresses']).__name__} ({data['addresses']!r})")
        if "contactInformations" in data and not isinstance(data["contactInformations"], list):
            errors.append(f"contactInformations must be a list, got {type(data['contactInformations']).__name__} ({data['contactInformations']!r})")
        if errors:
            raise TypeError("; ".join(errors))
        return data

    @model_validator(mode="after")
    def check_fields(self):
        # Company or person?
        if self.typeId == -30:  # Company
            # Only lastName (company name) required
            if not self.lastName:
                raise ValueError("lastName (company name) required for companies.")
        else:
            # Person: salutationId, firstName, lastName required
            missing = []
            if not self.salutationId:
                missing.append("salutationId")
            if not self.firstName:
                missing.append("firstName")
            if not self.lastName:
                missing.append("lastName")
            if missing:
                raise ValueError(f"Missing required fields for person: {', '.join(missing)}")
        return self


# print(json.dumps(PaymentProvider.model_json_schema(), indent=2))

## Testing Payment

In [189]:


try:
    ## Testing Credit Card
    provider = PaymentProvider(providerType=-160, providerName="AMEX", accountNumber="1234-4567", accountOwner="Jane Doe", expirationDate="1.1.2026")
    # PaymentProvider(providerType=-160, providerName="AMEX", accountNumber="1234-4567", accountOwner="Jane Doe")
    ## Testing Bank
    # PaymentProvider(providerType=-161, providerNumber="BIC123", providerName="MyBank", accountNumber="DE123456789", accountOwner="Jane Doe")
    # PaymentProvider(providerType=-161, accountNumber="DE123456789", accountOwner="Jane Doe")
    # PaymentProvider(providerType=-161, accountNumber="DE123456789")
    ## Testing PayPal
    # PaymentProvider(providerType=-163, accountNumber="a@bc.de")
    # PaymentProvider(providerType=-161)
    ## Testing illegal payment provider
    # PaymentProvider(providerType=-166, accountNumber="DE123456789")
    print(dict(provider))

except ValidationError as e:
    print(e)


{'providerType': -160, 'providerNumber': None, 'providerName': 'AMEX', 'accountNumber': '1234-4567', 'accountOwner': 'Jane Doe', 'expirationDate': '1.1.2026'}


## Testing Person

In [77]:
try:
    ## Correct info for corporate account 
    # Person(
    #     typeId=-30,
    #     lastName="ACME GmbH",
    #     addresses=[Address(typeId=-11, street="Hauptstr.", streetNumber="1", zipcode="12345", city="Berlin", countryCode="DE")],
    #     contactInformations=[ContactInfo(typeId=-23, value="info@acme.de")]
    # )
    ## Incorrect info for corporate account (country code missing in address)
    # Person(
    #     typeId=-30,
    #     lastName="ACME GmbH",
    #     addresses=[Address(typeId=-11, street="Hauptstr.", streetNumber="1", zipcode="12345", city="Berlin")],
    #     contactInformations=[ContactInfo(typeId=-23, value="info@acme.de")]
    # )
    ## Correct info for private account 
    # Person(
    #     typeId=-2101,
    #     salutationId=-31,
    #     firstName="Max",
    #     lastName="Mustermann",
    #     addresses=[Address(typeId=-10, street="Musterweg", streetNumber="2", zipcode="54321", city="Musterstadt", countryCode="DE")],
    #     contactInformations=[ContactInfo(typeId=-23, value="max@mustermann.de")]
    # )    
    ## Incorrect info for private account (addresses no a list)
    Person(
        typeId=-2101,
        salutationId=-31,
        firstName="Max",
        lastName="Mustermann",
        addresses=Address(typeId=-10, street="Musterweg", streetNumber="2", zipcode="54321", city="Musterstadt", countryCode="DE"),
        contactInformations=[ContactInfo(typeId=-23, value="max@mustermann.de")]
    )
except ValidationError as e:
    print(e)
except TypeError as e:
    print(e)

addresses must be a list, got Address (Address(typeId=-10, street='Musterweg', streetNumber='2', addition=None, zipcode='54321', city='Musterstadt', countryCode='DE'))


## Defining Attributes

In [110]:
from typing import List, Optional, Union, Literal
from pydantic import BaseModel, Field, create_model

def generate_attribute_model(attributes):
    fields = {}

    for attr in attributes:

        if not (attr.get("inputField") is True and attr.get("hide") is False):
            continue
        
        field_name = str(attr.get("attributeId"))
        description = attr.get("displayName", "")
        attr_type = attr.get("type")
        required = attr.get("required", False)
        default = attr.get("value")

        # Determine the field type
        if attr_type == "Liste":
            # Add enum values to description
            if attr.get('listelement'):
                enum_values = ', '.join([val['name'] for val in attr.get('listelement')])
                description += f", enum values: {enum_values}"
            field_type = str
        elif attr_type == 'Zahl':
            field_type = int
        elif attr_type == 'Datum':
            field_type = str
            description += ", pattern: ^\\d{{4}}-\\d{{2}}-\\d{{2}}$"
        else:
            field_type = str

        # Handle optionality
        if not required:
            field_type = Optional[field_type]

        fields[field_name] = (field_type, Field(default, description=description))

    # Dynamically create model
    DynamicAttributes = create_model("DynamicAttributes", **fields)
    return DynamicAttributes


In [174]:
from pydantic import BaseModel, Field, create_model, constr, conint
from typing import Optional, Literal, Union
import re

from typing import Optional, Literal, get_args
from pydantic import create_model, Field, constr

def generate_attribute_model(model_name, attributes):
    fields = {}

    for attr in attributes:
        if not (attr.get("inputField") is True and attr.get("hide") is False):
            continue

        # Valid Python field name
        field_name = f'attribute_{attr["attributeId"]}'

        title = attr.get("displayName", "")
        description = ""
        attr_type = attr.get("type")
        required = attr.get("required", False)
        default = attr.get("value", None)
        
        # Choose field type and constraints
        if attr_type == "Liste":
            allowed = tuple(val['name'] for val in attr.get("listelement", []))
            field_type = Literal[allowed] if allowed else str
            description += f" (allowed: {', '.join(allowed)})"
        elif attr_type == "Zahl":
            field_type = int
            try:
              if default is not None:
                default= int(default)
            except:
                pass
            description += " (must be integer)"
        elif attr_type == "Datum":
            field_type = constr(pattern=r'^\d{4}-\d{2}-\d{2}$')
            description += " (YYYY-MM-DD)"
        elif attr_type == "Text":
            field_type = str
        elif attr_type == "Auswahl":
            field_type = bool
            try:
              if default is not None and isinstance(default, str):
                  default = (default.lower() == "true")
            except:
                pass
        else:
            field_type = str

        if not required:
            field_type = Optional[field_type]
            # Default to None if not set
            default = default if default is not None else None
        else:
            default = default if default is not None else ...

        fields[field_name] = (field_type, Field(title=title, description=description, default=default))

    return create_model(model_name, **fields)

In [155]:
print(json.dumps(data[1]['attributecategorie'],indent=2))

[
  {
    "name": "Allgemeine Kategorie",
    "displayName": "Allgemeine Kategorie",
    "rank": 30,
    "expandable": false,
    "collapse": null,
    "header": "",
    "footer": "",
    "displayAttributeCategoryDtoMap": {
      "Diebstahl": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "SelfActive Erfassung": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "Ger\u00e4teimport": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "Provisionierung": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "Vertrag": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "Vertragsvorlage": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "Beitragsberechnung": {
        "collapse": null,
        "header": "",
        "footer": ""
      },
      "TouchActive Extra": {
        "collaps

In [175]:
# attributes = data[1]['attributecategorie'][0]['attribute']
attributes = [
    attr
    for cat in data[1]['attributecategorie']
    for attr in cat['attribute']
]

attribute_model = generate_attribute_model('attributes', attributes)
print(json.dumps(attribute_model.model_json_schema(),indent=2))

{
  "properties": {
    "attribute_331": {
      "default": "Handy/Smartphone",
      "description": " (allowed: Handy/Smartphone, Tablet, Smartwatch)",
      "enum": [
        "Handy/Smartphone",
        "Tablet",
        "Smartwatch"
      ],
      "title": "Ger\u00e4tetyp",
      "type": "string"
    }
  },
  "title": "attributes",
  "type": "object"
}


In [176]:
print("\nValid case:")
attribute = attribute_model(
    attribute_331="Tablet",
    attribute_555=123456,
    attribute_777="2024-07-25"
)
print(attribute.model_dump_json(indent=2))


Valid case:
{
  "attribute_331": "Tablet"
}


In [177]:
modules = []
for sub_prod in data[1]["subproduct"]:
    ppv_id = sub_prod["partProductVariantId"]
    if not ppv_id in [19, 3112, 3091]:
        modules.extend(
            # sub_prod["productTreeRenderData"]
            [
                attr
                for cat in sub_prod["productTreeRenderData"]['attributecategorie']
                for attr in cat['attribute']
            ]
        )
# print(json.dumps(modules,indent=2))
module_model = generate_attribute_model('modules', modules)
print(json.dumps(module_model.model_json_schema(),indent=2))

{
  "properties": {
    "attribute_604": {
      "anyOf": [
        {
          "type": "boolean"
        },
        {
          "type": "null"
        }
      ],
      "default": false,
      "description": "",
      "title": "Ich kenne meine IMEI-/Seriennummer noch nicht"
    },
    "attribute_67": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": "",
      "description": "",
      "title": "IMEI-/Seriennummer"
    },
    "attribute_3730": {
      "default": "501 - 750 \u20ac",
      "description": " (allowed: 0 - 250 \u20ac, 251 - 500 \u20ac, 501 - 750 \u20ac, 751 - 1.000 \u20ac, 1.001 - 1.250 \u20ac, 1.251 - 1.500 \u20ac, 1.501 - 1.750 \u20ac, 1.751 - 2.000 \u20ac, 2.001 - 2.500 \u20ac, 2.501 - 3.000 \u20ac)",
      "enum": [
        "0 - 250 \u20ac",
        "251 - 500 \u20ac",
        "501 - 750 \u20ac",
        "751 - 1.000 \u20ac",
        "1.001 - 1.250 \u20ac",
        "1.251 - 1.500 \u20a

In [180]:
print("\nValid case:")
module = module_model(
    attribute_331="Tablet",
    attribute_555=123456,
    attribute_777="2024-07-25",
    attribute_70="Apple"
)
print(module.model_dump_json(indent=2))


Valid case:
{
  "attribute_604": false,
  "attribute_67": "",
  "attribute_3730": "501 - 750 €",
  "attribute_3733": "bis 2 Wochen",
  "attribute_70": "Apple",
  "attribute_217": "",
  "attribute_130": "",
  "attribute_16": 24
}


In [179]:
print(module_model.model_json_schema()["required"])

['attribute_70']


In [111]:
attributes = data[1]['attributecategorie'][0]['attribute']
print(json.dumps(attributes, indent=2))
# data[1]['attributecategorie'][0]['attribute']

[
  {
    "attributeId": 331,
    "productVariantAttributeId": 12853,
    "name": "Ger\u00e4tetyp (privat)",
    "displayName": "Ger\u00e4tetyp",
    "displayDescription": "",
    "catName": "Allgemeine Kategorie",
    "catDisplayName": "Allgemeine Kategorie",
    "catRank": 30,
    "attributeCategoryId": -1,
    "placeholder": null,
    "description": "",
    "type": "Liste",
    "typeId": -102014,
    "value": "Handy/Smartphone",
    "inputField": true,
    "hide": false,
    "advanced": false,
    "dynamicLoaderId": null,
    "productVariantId": 3415,
    "productName": "HandySchutzbrief Basis Check24",
    "personAttributeTypeId": null,
    "documentAttributeTypeId": null,
    "businessObjectAttributeTypeId": null,
    "required": true,
    "rank": 100,
    "listId": 25,
    "properties": {},
    "autocomplete": null,
    "autofocus": null,
    "readonly": null,
    "size": null,
    "min": null,
    "max": null,
    "minDate": null,
    "maxDate": null,
    "minLength": null,
    

In [112]:
attribute_model = generate_attribute_model(attributes)

In [113]:
# print(attribute_model)
print(json.dumps(attribute_model.model_json_schema(), indent=2))

{
  "properties": {
    "331": {
      "default": "Handy/Smartphone",
      "description": "Ger\u00e4tetyp, enum values: Handy/Smartphone, Tablet, Smartwatch",
      "title": "331",
      "type": "string"
    }
  },
  "title": "DynamicAttributes",
  "type": "object"
}


In [120]:
inst = attribute_model(attributeId=331, value='Tablet')
print(inst)

331='Handy/Smartphone'


# load test data for form submit

In [17]:
with open("test_data.json", "r", encoding="utf-8") as file:  
    test_data = json.load(file)

print(json.dumps(test_data, indent=2, ensure_ascii=False))


{
  "productDto": {
    "productVariantId": 3571,
    "removed": false,
    "attributes": [
      {
        "id": 15991,
        "value": "500011"
      },
      {
        "id": 15994,
        "value": "true"
      },
      {
        "id": 15997,
        "value": "true"
      },
      {
        "id": 16000,
        "value": "true"
      },
      {
        "id": 16003,
        "value": "false"
      },
      {
        "id": 16006,
        "value": "true"
      },
      {
        "id": 16009,
        "value": "true"
      },
      {
        "id": 16012,
        "value": "false"
      },
      {
        "id": 15988,
        "value": "true"
      },
      {
        "id": 16051,
        "value": "true"
      },
      {
        "id": 16045,
        "value": "true"
      },
      {
        "id": 16048,
        "value": "true"
      }
    ],
    "subproducts": [
      {
        "productVariantId": 3091,
        "modules": [
          {
            "productVariantId": 3091,
            "removed

In [22]:
url = "https://test-protactive.schutzgarant.de/ca-rest/import"
params = {"dryRun": "true", "async": "true"}
headers = {"Accept": "application/json", "Content-Type": "application/json"}
session = requests.Session()
answer = session.post(
    url,
    auth=HTTPBasicAuth(user, pwd),
    headers=headers,
    params=params,
    json=test_data,
)

try:
    content = answer.json()
except Exception:
    content = answer.text
response = {"status_code": answer.status_code, "content": content}

In [24]:
response

{'status_code': 400,
 'content': 'Unrecognized field &quot;productDto&quot; (class at.intervista.collactive.rest.external.model.Contract), not marked as ignorable'}